In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np

# This line tells the notebook to show plots inside of the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sb
import seaborn.apionly as sns

In [2]:
#Get counties
dp05 = pd.read_csv('files/ACS_14_5YR_DP05/ACS_14_5YR_DP05_with_ann.csv',skiprows=1)
counties = dp05.Geography
counties=counties.apply(lambda x: x.replace(',','').replace(' ','+'))

In [ ]:
#Scrape the site
url_search = 'https://livabilityindex.aarp.org/search#'
result=[]

for i in range(0,len(counties)):
    try:
        url = url_search + counties[i]
        driver = webdriver.Chrome()
        driver.implicitly_wait(1)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)
    finally:
        driver.quit()

    #Glean the variables
    housing = soup.find('div',class_='icon-housing livability-score-detail').get_text()
    neighborhood = soup.find('div',class_='icon-neighborhood livability-score-detail').get_text()
    transportation = soup.find('div',class_='icon-transportation livability-score-detail').get_text()
    environment = soup.find('div',class_='icon-environment livability-score-detail').get_text()
    health = soup.find('div',class_='icon-health livability-score-detail').get_text()
    engagement = soup.find('div',class_='icon-engagement livability-score-detail').get_text()
    opportunity = soup.find('div',class_='icon-opportunity livability-score-detail').get_text()
    
    #Create data frame
    temp = {'country':counties[i],'housing':housing,'neighborhood':neighborhood,'transporation':transportation,'environment':environment,
           'health':health,'engagement':engagement,'opportunity':opportunity}
    temp_df = pd.DataFrame(temp,index=[counties[i]])
    
    #append
    result.append(temp_df)


In [ ]:
#concatenate and save results    
result = pd.concat(result,axis=0)
result.to_csv('files/livability.csv')

In [3]:
#Control variables
#exclude county names
controls = pd.read_csv('files/livability.csv').iloc[:,1:]
controls.head()

,engagement,environment,health,housing,neighborhood,opportunity,transportation
Autauga+County+Alabama,64.0,57.0,34.0,58.0,37.0,41.0,34.0
Baldwin+County+Alabama,64.0,60.0,48.0,54.0,36.0,40.0,44.0
Barbour+County+Alabama,52.0,24.0,15.0,66.0,32.0,34.0,47.0
Bibb+County+Alabama,34.0,57.0,17.0,43.0,31.0,53.0,34.0
Blount+County+Alabama,42.0,59.0,29.0,44.0,37.0,65.0,34.0
